In [ ]:
!pip install -q contractions transformers sent2vec
!pip install -q imbalanced-learn
!pip install -q transformers
!pip install -q sent2vec

     |████████████████████████████████| 4.0 MB 4.3 MB/s 
     |████████████████████████████████| 287 kB 39.3 MB/s 
     |████████████████████████████████| 106 kB 75.8 MB/s 
     |████████████████████████████████| 6.6 MB 42.4 MB/s 
     |████████████████████████████████| 596 kB 64.1 MB/s 
     |████████████████████████████████| 84 kB 3.6 MB/s 
     |████████████████████████████████| 880 kB 87.9 MB/s 


In [ ]:
!pip install -q spacy-streamlit

     |████████████████████████████████| 6.2 MB 4.2 MB/s 
     |████████████████████████████████| 10.1 MB 65.1 MB/s 
     |████████████████████████████████| 181 kB 23.3 MB/s 
     |████████████████████████████████| 457 kB 53.0 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
     |████████████████████████████████| 653 kB 49.4 MB/s 
     |████████████████████████████████| 10.1 MB 11.5 MB/s 
     |████████████████████████████████| 58 kB 6.1 MB/s 
     |████████████████████████████████| 4.3 MB 24.9 MB/s 
     |████████████████████████████████| 164 kB 4.9 MB/s 
     |████████████████████████████████| 76 kB 5.1 MB/s 
     |████████████████████████████████| 181 kB 52.4 MB/s 
     |████████████████████████████████| 111 kB 53.6 MB/s 
     |████████████████████████████████| 63 kB 1.2 MB/s 
     |████████████████████████████████| 131 kB 46.5 MB/s 
     |████████████████████████████████| 428 kB 37.7 MB/s 
     |████████████████████████████████| 130 kB 55.5 MB/s 
     |████████████████

In [ ]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.8 MB 4.1 MB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


Strategy 2: CREATE CUSTOM NERS

In [ ]:
import pandas as pd
import spacy
import numpy as np
from spacy import displacy
import re
import nltk
import contractions
import warnings

train = pd.read_csv('task3_training.tsv',sep='\t')
valid = pd.read_csv('task3_validation.tsv',sep='\t')
train.dropna(axis=1, how='all',inplace=True)
valid.dropna(axis=1, how='all',inplace=True)

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
# !python -m spacy download en_core_web_sm
nlp=spacy.load('en_core_web_sm')
ner=nlp.get_pipe("ner")

In [ ]:
#@title Helper function for cleaning tweets

def remove_tags_mentions(text):
    pattern = re.compile(r'(@\S+|#\S+)')
    return pattern.sub('', text)

def remove_redundant_chars(text):
    pattern = re.compile(r'(&gt|&amp|&lt|wtf|fuck\'n|®|©|rt)')
    return pattern.sub('', text)

# Reference: https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

EMOTICONS = {
    u":‑\)":"Happy face or smiley",
    u":\)":"Happy face or smiley",
    u":-\]":"Happy face or smiley",
    u":\]":"Happy face or smiley",
    u":-3":"Happy face smiley",
    u":3":"Happy face smiley",
    u":->":"Happy face smiley",
    u":>":"Happy face smiley",
    u"8-\)":"Happy face smiley",
    u":o\)":"Happy face smiley",
    u":-\}":"Happy face smiley",
    u":\}":"Happy face smiley",
    u":-\)":"Happy face smiley",
    u":c\)":"Happy face smiley",
    u":\^\)":"Happy face smiley",
    u"=\]":"Happy face smiley",
    u"=\)":"Happy face smiley",
    u":‑D":"Laughing, big grin or laugh with glasses",
    u":D":"Laughing, big grin or laugh with glasses",
    u"8‑D":"Laughing, big grin or laugh with glasses",
    u"8D":"Laughing, big grin or laugh with glasses",
    u"X‑D":"Laughing, big grin or laugh with glasses",
    u"XD":"Laughing, big grin or laugh with glasses",
    u"=D":"Laughing, big grin or laugh with glasses",
    u"=3":"Laughing, big grin or laugh with glasses",
    u"B\^D":"Laughing, big grin or laugh with glasses",
    u":-\)\)":"Very happy",
    u":‑\(":"Frown, sad, andry or pouting",
    u":-\(":"Frown, sad, andry or pouting",
    u":\(":"Frown, sad, andry or pouting",
    u":‑c":"Frown, sad, andry or pouting",
    u":c":"Frown, sad, andry or pouting",
    u":‑<":"Frown, sad, andry or pouting",
    u":<":"Frown, sad, andry or pouting",
    u":‑\[":"Frown, sad, andry or pouting",
    u":\[":"Frown, sad, andry or pouting",
    u":-\|\|":"Frown, sad, andry or pouting",
    u">:\[":"Frown, sad, andry or pouting",
    u":\{":"Frown, sad, andry or pouting",
    u":@":"Frown, sad, andry or pouting",
    u">:\(":"Frown, sad, andry or pouting",
    u":'‑\(":"Crying",
    u":'\(":"Crying",
    u":'‑\)":"Tears of happiness",
    u":'\)":"Tears of happiness",
    u"D‑':":"Horror",
    u"D:<":"Disgust",
    u"D:":"Sadness",
    u"D8":"Great dismay",
    u"D;":"Great dismay",
    u"D=":"Great dismay",
    u"DX":"Great dismay",
    u":‑O":"Surprise",
    u":O":"Surprise",
    u":‑o":"Surprise",
    u":o":"Surprise",
    u":-0":"Shock",
    u"8‑0":"Yawn",
    u">:O":"Yawn",
    u":-\*":"Kiss",
    u":\*":"Kiss",
    u":X":"Kiss",
    u";‑\)":"Wink or smirk",
    u";\)":"Wink or smirk",
    u"\*-\)":"Wink or smirk",
    u"\*\)":"Wink or smirk",
    u";‑\]":"Wink or smirk",
    u";\]":"Wink or smirk",
    u";\^\)":"Wink or smirk",
    u":‑,":"Wink or smirk",
    u";D":"Wink or smirk",
    u":‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"X‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"XP":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"d:":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"=p":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u">:P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":-[.]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":S":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":‑\|":"Straight face",
    u":\|":"Straight face",
    u":$":"Embarrassed or blushing",
    u":‑x":"Sealed lips or wearing braces or tongue-tied",
    u":x":"Sealed lips or wearing braces or tongue-tied",
    u":‑#":"Sealed lips or wearing braces or tongue-tied",
    u":#":"Sealed lips or wearing braces or tongue-tied",
    u":‑&":"Sealed lips or wearing braces or tongue-tied",
    u":&":"Sealed lips or wearing braces or tongue-tied",
    u"O:‑\)":"Angel, saint or innocent",
    u"O:\)":"Angel, saint or innocent",
    u"0:‑3":"Angel, saint or innocent",
    u"0:3":"Angel, saint or innocent",
    u"0:‑\)":"Angel, saint or innocent",
    u"0:\)":"Angel, saint or innocent",
    u":‑b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"0;\^\)":"Angel, saint or innocent",
    u">:‑\)":"Evil or devilish",
    u">:\)":"Evil or devilish",
    u"\}:‑\)":"Evil or devilish",
    u"\}:\)":"Evil or devilish",
    u"3:‑\)":"Evil or devilish",
    u"3:\)":"Evil or devilish",
    u">;\)":"Evil or devilish",
    u"\|;‑\)":"Cool",
    u"\|‑O":"Bored",
    u":‑J":"Tongue-in-cheek",
    u"#‑\)":"Party all night",
    u"%‑\)":"Drunk or confused",
    u"%\)":"Drunk or confused",
    u":-###..":"Being sick",
    u":###..":"Being sick",
    u"<:‑\|":"Dump",
    u"\(>_<\)":"Troubled",
    u"\(>_<\)>":"Troubled",
    u"\(';'\)":"Baby",
    u"\(\^\^>``":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(\^_\^;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(~_~;\) \(・\.・;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-\)zzz":"Sleeping",
    u"\(\^_-\)":"Wink",
    u"\(\(\+_\+\)\)":"Confused",
    u"\(\+o\+\)":"Confused",
    u"\(o\|o\)":"Ultraman",
    u"\^_\^":"Joyful",
    u"\(\^_\^\)/":"Joyful",
    u"\(\^O\^\)／":"Joyful",
    u"\(\^o\^\)／":"Joyful",
    u"\(__\)":"Kowtow as a sign of respect, or dogeza for apology",
    u"_\(\._\.\)_":"Kowtow as a sign of respect, or dogeza for apology",
    u"<\(_ _\)>":"Kowtow as a sign of respect, or dogeza for apology",
    u"<m\(__\)m>":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(__\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(_ _\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"\('_'\)":"Sad or Crying",
    u"\(/_;\)":"Sad or Crying",
    u"\(T_T\) \(;_;\)":"Sad or Crying",
    u"\(;_;":"Sad of Crying",
    u"\(;_:\)":"Sad or Crying",
    u"\(;O;\)":"Sad or Crying",
    u"\(:_;\)":"Sad or Crying",
    u"\(ToT\)":"Sad or Crying",
    u";_;":"Sad or Crying",
    u";-;":"Sad or Crying",
    u";n;":"Sad or Crying",
    u";;":"Sad or Crying",
    u"Q\.Q":"Sad or Crying",
    u"T\.T":"Sad or Crying",
    u"QQ":"Sad or Crying",
    u"Q_Q":"Sad or Crying",
    u"\(-\.-\)":"Shame",
    u"\(-_-\)":"Shame",
    u"\(一一\)":"Shame",
    u"\(；一_一\)":"Shame",
    u"\(=_=\)":"Tired",
    u"\(=\^\·\^=\)":"cat",
    u"\(=\^\·\·\^=\)":"cat",
    u"=_\^=	":"cat",
    u"\(\.\.\)":"Looking down",
    u"\(\._\.\)":"Looking down",
    u"\^m\^":"Giggling with hand covering mouth",
    u"\(\・\・?":"Confusion",
    u"\(?_?\)":"Confusion",
    u">\^_\^<":"Normal Laugh",
    u"<\^!\^>":"Normal Laugh",
    u"\^/\^":"Normal Laugh",
    u"\（\*\^_\^\*）" :"Normal Laugh",
    u"\(\^<\^\) \(\^\.\^\)":"Normal Laugh",
    u"\(^\^\)":"Normal Laugh",
    u"\(\^\.\^\)":"Normal Laugh",
    u"\(\^_\^\.\)":"Normal Laugh",
    u"\(\^_\^\)":"Normal Laugh",
    u"\(\^\^\)":"Normal Laugh",
    u"\(\^J\^\)":"Normal Laugh",
    u"\(\*\^\.\^\*\)":"Normal Laugh",
    u"\(\^—\^\）":"Normal Laugh",
    u"\(#\^\.\^#\)":"Normal Laugh",
    u"\（\^—\^\）":"Waving",
    u"\(;_;\)/~~~":"Waving",
    u"\(\^\.\^\)/~~~":"Waving",
    u"\(-_-\)/~~~ \($\·\·\)/~~~":"Waving",
    u"\(T_T\)/~~~":"Waving",
    u"\(ToT\)/~~~":"Waving",
    u"\(\*\^0\^\*\)":"Excited",
    u"\(\*_\*\)":"Amazed",
    u"\(\*_\*;":"Amazed",
    u"\(\+_\+\) \(@_@\)":"Amazed",
    u"\(\*\^\^\)v":"Laughing,Cheerful",
    u"\(\^_\^\)v":"Laughing,Cheerful",
    u"\(\(d[-_-]b\)\)":"Headphones,Listening to music",
    u'\(-"-\)':"Worried",
    u"\(ーー;\)":"Worried",
    u"\(\^0_0\^\)":"Eyeglasses",
    u"\(\＾ｖ\＾\)":"Happy",
    u"\(\＾ｕ\＾\)":"Happy",
    u"\(\^\)o\(\^\)":"Happy",
    u"\(\^O\^\)":"Happy",
    u"\(\^o\^\)":"Happy",
    u"\)\^o\^\(":"Happy",
    u":O o_O":"Surprised",
    u"o_0":"Surprised",
    u"o\.O":"Surpised",
    u"\(o\.o\)":"Surprised",
    u"oO":"Surprised",
    u"\(\*￣m￣\)":"Dissatisfied",
    u"\(‘A`\)":"Snubbed or Deflated"
}

def remove_emoticons(text):
    emoticons_pattern = re.compile(u'(' + u'|'.join(emo for emo in EMOTICONS) + u')')
    return emoticons_pattern.sub(r'', text)

def expand_contractions(text):
    return contractions.fix(text)

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCTUATIONS))

def remove_whitespaces(text):
    pattern = re.compile(r" +")
    return pattern.sub(" ", text).strip()

def remove_repeated_chars(text):
    return re.sub("(.)\\1{2,}", "\\1", text)

def remove_single_char(text):
    pattern = re.compile(r' ([a-zA-Z]{1}) ')
    return pattern.sub(" ", text)

def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in STOPWORDS])

def clean_tweets(df, col):
    
    # Lower case
    df[col] = df[col].str.lower()

    # Remove URL's
    df[col] = df[col].str.replace(r'https?://\S+|www\.\S+', '', regex=True)
    
    # Remove Mentions and Hastags
    df[col] = df[col].apply(lambda x: remove_tags_mentions(x))

    # Remove redundant characters
    df[col] = df[col].apply(lambda x: remove_redundant_chars(x))

    # Remove emoji's
    df[col] = df[col].apply(lambda x: remove_emoji(x))

    # Remove emoticons
    df[col] = df[col].apply(lambda x: remove_emoticons(x))
    
    # Expand contractions
    df[col] = df[col].apply(lambda x: expand_contractions(x))
    
    # Remove stopwords
    df[col] = df[col].apply(lambda x: remove_stopwords(x))

    # Remove punctuations
    df[col] = df[col].apply(lambda x: remove_punctuation(x))

    # Remove repeated characters in tweet
    df[col] = df[col].apply(lambda x: remove_repeated_chars(x))

    # Remove single character within text
    df[col] = df[col].apply(lambda x: remove_single_char(x))

    # Remove extract whitespaces within tweet
    df[col] = df[col].apply(lambda x: remove_whitespaces(x))

    # Drop duplicate rows
    df.drop_duplicates(subset=['tweet'], inplace=True, keep='first')
    
    return df

In [ ]:
# train = clean_tweets(train, 'tweet')
# valid = clean_tweets(valid, 'tweet')

TRAIN_DATA = []
for row in train.itertuples():
    if str(row[2]) != 'nan':
        # pass
        TRAIN_DATA.append((row[7],{
            'entities':[(int(row[2]),int(row[3]),"CSL")]
        }))
    # else:
    #     print(row)
    #     break

In [ ]:
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [ ]:
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.training import Example
with nlp.disable_pipes(*unaffected_pipes):
    for iteration in range(50):
        print(f'Iteration {iteration}')
        random.shuffle(TRAIN_DATA) 
        losses = {}
        batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            
            for text, annotations in batch:
                
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                texts, annotations = zip(*batch)
                nlp.update([example], losses=losses, drop=0.2)

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19
Iteration 20
Iteration 21
Iteration 22
Iteration 23
Iteration 24
Iteration 25
Iteration 26
Iteration 27
Iteration 28
Iteration 29
Iteration 30
Iteration 31
Iteration 32
Iteration 33
Iteration 34
Iteration 35
Iteration 36
Iteration 37
Iteration 38
Iteration 39
Iteration 40
Iteration 41
Iteration 42
Iteration 43
Iteration 44
Iteration 45
Iteration 46
Iteration 47
Iteration 48
Iteration 49


In [ ]:
main_list = []
for tweet in valid['tweet']:
    doc = nlp(tweet)
    main_list.append([ent.text for ent in doc.ents]) 
    # print("Entities", )


In [ ]:
valid['prediction_extractions']= main_list

In [ ]:
valid.to_csv('predicted_with_data.csv',sep='|')

In [ ]:
valid

,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term,prediction_extractions
0,332317478170546176,28.0,37.0,ADR,allergies,avelox,"do you have any medication allergies? ""asthma!!!"" me: ""........"" pt: ""no wait. avelox, that's it!"" ""so no other allergies?"" ""right!"" *cont",10013661.0,drug allergy,[]
1,347806215776116737,31.0,46.0,ADR,HURT YOUR Liver,avelox,"@ashleylvivian if #avelox has hurt your liver, avoid tylenol always, as it further damages liver, eat grapefruit unless taking cardiac drugs",10024668.0,liver damage,[]
2,350336129817509888,48.0,50.0,ADR,AD,baclofen,"apparently, baclofen greatly exacerbates the ""ad"" part of my adhd. average length of focus today: about 30 seconds.",10003731.0,attention deficit disorder,[]
3,350336129817509888,88.0,93.0,ADR,focus,baclofen,"apparently, baclofen greatly exacerbates the ""ad"" part of my adhd. average length of focus today: about 30 seconds.",10003738.0,attention impaired,[]
4,332540699692130304,11.0,15.0,ADR,died,cipro,pt of mine died from cipro rt @ciproispoison: @gastromom if only more doctors thought like you! i lost my entire life to 12 cipro pills,10011906.0,death,[died]
5,332542581760204800,41.0,45.0,ADR,died,cipro,"@gastromom the only pt of mine who ever died was one age 21 profound autism chronic underwt &amp; gi issues, given lots of cipro. terrible",10011906.0,death,[]
6,343944308828811264,66.0,79.0,ADR,tendon damage,cipro,owww. i hurt my foot. &amp; am concerned. 1st warning on cipro is tendon damage. :-/ it certainly wasn't a rupture. i'm sure i'm just paranoid.,10043242.0,tendon injury,[]
7,346700255829561344,NaN,NaN,NaN,NaN,cipro,@deadfanclub @danadelany i have a penicillin/ keflex allergy so my dr writes me 10 days of zpack or i'd still be sick too! sometimes cipro,NaN,NaN,[sick]
8,348120419921952770,NaN,NaN,NaN,NaN,cipro,@joshuagates you forgot the immodium for when the cipro kills all the good bacteria. :) #travelskills,NaN,NaN,[]
9,348644040444637184,26.0,32.0,ADR,dreams,Robitussin,@bilgeebiri oh robitussin dreams are notorious. although what will really turn 'em into a bunuel movie is cipro.,10004969.0,bizarre dreams,[]
